# Motivation
Companies need to assign people and resources to tasks at specific times. Schedule employees in multiple shifts, subject to a complex set of constraints and staffing requirements.

# Introduction to PuLP
Linear programming (LP) is one of the best methods to find optimal solutions for problems with constraints like the above. PuLP is a Python library that makes it easy to apply linear programming using Python.

In [1]:
import pandas as pd
import pulp
from itertools import product
from termcolor import colored

In [2]:
d = {
    'day': [2, 3, 4, 5, 6, 7, 8],
    'employee': ['A', 'B', 'C', 'D'],
    'shift': ['S1', 'S2'],
    'S1': 2,
    'S2': 1,
    'each employee have to work (day)': 5
}

d

{'day': [2, 3, 4, 5, 6, 7, 8],
 'employee': ['A', 'B', 'C', 'D'],
 'shift': ['S1', 'S2'],
 'S1': 2,
 'S2': 1,
 'each employee have to work (day)': 5}

## Transform into pulp library

In [3]:
# dynamic variables
lst = [(d, s, e) for d, s, e in product(d['day'], d['shift'], d['employee'])]
lst[:3]

[(2, 'S1', 'A'), (2, 'S1', 'B'), (2, 'S1', 'C')]

In [4]:
# minimize problem
x = pulp.LpVariable.dicts("x", lst, cat='Binary')
model = pulp.LpProblem("scheduling", pulp.LpMinimize)

# objective
model += sum([x[i]for i in lst])

# shift per day
for day in d['day']:
    if day in [2, 3, 4, 5, 6]:
        for s in d['shift']:
            model += sum(x[i] for i in lst if {s, day} <= set(i)) >= d[s]

# total working days
for e in d['employee']:
    model += sum(x[i] for i in lst if {e} <= set(i)) == 5

# only work 1 shift per pax
for day in d['day']:
    for e in d['employee']:
        model += sum(x[i] for i in lst if {e, 'S1', day} <= set(i) or {e, 'S2', day} <= set(i)) <= 1

You can double check what you have written to your model as aligned with your fomulation

In [5]:
model

scheduling:
MINIMIZE
1*x_(2,_'S1',_'A') + 1*x_(2,_'S1',_'B') + 1*x_(2,_'S1',_'C') + 1*x_(2,_'S1',_'D') + 1*x_(2,_'S2',_'A') + 1*x_(2,_'S2',_'B') + 1*x_(2,_'S2',_'C') + 1*x_(2,_'S2',_'D') + 1*x_(3,_'S1',_'A') + 1*x_(3,_'S1',_'B') + 1*x_(3,_'S1',_'C') + 1*x_(3,_'S1',_'D') + 1*x_(3,_'S2',_'A') + 1*x_(3,_'S2',_'B') + 1*x_(3,_'S2',_'C') + 1*x_(3,_'S2',_'D') + 1*x_(4,_'S1',_'A') + 1*x_(4,_'S1',_'B') + 1*x_(4,_'S1',_'C') + 1*x_(4,_'S1',_'D') + 1*x_(4,_'S2',_'A') + 1*x_(4,_'S2',_'B') + 1*x_(4,_'S2',_'C') + 1*x_(4,_'S2',_'D') + 1*x_(5,_'S1',_'A') + 1*x_(5,_'S1',_'B') + 1*x_(5,_'S1',_'C') + 1*x_(5,_'S1',_'D') + 1*x_(5,_'S2',_'A') + 1*x_(5,_'S2',_'B') + 1*x_(5,_'S2',_'C') + 1*x_(5,_'S2',_'D') + 1*x_(6,_'S1',_'A') + 1*x_(6,_'S1',_'B') + 1*x_(6,_'S1',_'C') + 1*x_(6,_'S1',_'D') + 1*x_(6,_'S2',_'A') + 1*x_(6,_'S2',_'B') + 1*x_(6,_'S2',_'C') + 1*x_(6,_'S2',_'D') + 1*x_(7,_'S1',_'A') + 1*x_(7,_'S1',_'B') + 1*x_(7,_'S1',_'C') + 1*x_(7,_'S1',_'D') + 1*x_(7,_'S2',_'A') + 1*x_(7,_'S2',_'B') + 1*x_(7,_'S2',

In [6]:
# solve
model.solve(pulp.PULP_CBC_CMD(msg=False))
text = f'status: {pulp.LpStatus[model.status]}'
color = 'blue' if model.status == 1 else 'red'
print(colored(text, color, attrs=['reverse']))

status: Optimal


In [7]:
result = {i: int(x[i].varValue) for i in x}
report = pd.DataFrame.from_dict(result, orient='index').reset_index()
report.columns = ['var', 'value']
report[['day', 'shift', 'employee']] = pd.DataFrame(report['var'].tolist(), index=report.index)
report.sort_values(by='value', ascending=False, inplace=True)
report.query('value != 0', inplace=True)
report.drop(columns='var', inplace=True)

## Report
The result will show the number of racks, pallets needed for item A, B

In [8]:
report.pivot_table(values='value', index=['employee', 'shift'], columns='day')

day               2    3    4    5    6    7    8
employee shift                                   
A        S1     NaN  NaN  1.0  1.0  NaN  NaN  1.0
         S2     NaN  1.0  NaN  NaN  NaN  1.0  NaN
B        S1     1.0  1.0  NaN  1.0  1.0  NaN  NaN
         S2     NaN  NaN  NaN  NaN  NaN  NaN  1.0
C        S1     NaN  NaN  1.0  NaN  1.0  NaN  NaN
         S2     1.0  NaN  NaN  1.0  NaN  NaN  1.0
D        S1     1.0  1.0  NaN  NaN  NaN  1.0  NaN
         S2     NaN  NaN  1.0  NaN  1.0  NaN  NaN